In [1]:
import os
from pathlib import Path

In [2]:
# Domain Contract
os.environ['HADRON_PM_PATH'] = Path('${PWD}/metadata/contracts/workshop').as_posix() 

# local data stores
os.environ['HADRON_DEFAULT_PATH'] = Path('${PWD}/metadata/data/workshop').as_posix()
os.environ['HADRON_TRANSITION_INTEL_SERVICE_SOURCE_PATH'] = Path('${PWD}/metadata/source').as_posix()

In [3]:
from ds_discovery import Transition

In [4]:
tr = Transition.from_env("intel_service", has_contract=False)

In [5]:
tr.setup_bootstrap(domain='Telecom')
tr.set_source('clm_dump.xlsx', nrows=5000)
tr.set_persist()

In [6]:
df = tr.load_source_canonical()

In [7]:
# map the names
df = tr.tools.auto_clean_header(df, rename_map={
                'serviceRequests.customerInfo.serviceCode':'serviceCode',
                'serviceRequests.serviceRequestProcessingDate':'processingDate',
                'serviceRequests.serviceRequestReason.masterCode':'reason',
                'serviceRequests.requestor.requestorCode.masterCode':'requestor',
                'serviceRequests.serviceRequestType.masterCode':'type',
                'serviceRequests.serviceRequestCreationDate':'creationDate',
                'serviceRequests.serviceRequestStatus.masterCode':'status',
                'serviceRequests.serviceRequestSubtype.masterCode':'subType',
                'serviceRequests.subServiceType.masterCode':'subServiceType',
                'serviceRequests.businessType.masterCode':'businessType',
                'serviceRequests.serviceRequestNumber':'requestNumber',
                'serviceRequests.modifiedDate':'modifiedDate',
})

In [17]:
# Tidy up the columns (selection)
df = tr.tools.to_remove(df, headers=["businessType", "subServiceType"])
df = tr.tools.to_date_type(df, regex="Date", timezone='UTC')

In [18]:
tr.run_component_pipeline()

In [20]:
tr.canonical_report(tr.load_persist_canonical())